In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from tensorflow import cast,float32
from keras import backend as kb
from statistics import mean, stdev

2022-09-22 20:50:23.265015: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def generate_kmer_multiple(seqlist,k):
    kmer_list = []
    n = -1
    for seq in seqlist:
        kmer_list.append(generate_kmer_single(seq,k))
    return kmer_list
    
def generate_kmer_single(seq,k):
    kmer = ""
    for i in range(0,len(seq)-k,1):
        kmer += seq[i:i+k]+" "
    return kmer[:-1]

def test_rmse(model,X_test,Y_test):
    test_preds = model.predict(X_test)
    mse = mean_squared_error(Y_test, test_preds)
    rmse = sqrt(mse)
    return rmse

def root_mean_squared_error(y_true, y_pred):
    y_true = cast(y_true,float32)
    return kb.sqrt(kb.mean(kb.square(y_pred - y_true)))

In [3]:
def read_region(region):
    da = pd.read_csv("datasets/full_length_reads.csv")
    file_handle = open("datasets/"+region+".fasta","r")
    seq = []
    seqid = []
    tmp_seq = ""
    for line in file_handle:
        if (line[0] == ">"):
            if tmp_seq != "":
                seq.append(tmp_seq)
            seqid.append(line.split("\n")[0][1:])
            tmp_seq = ""
        else:
            tmp_seq+=line.split("\n")[0]
    seq.append(tmp_seq)
    file_handle.close()
    sub = pd.DataFrame([seq,seqid], index = [region,"accession"])
    sub = sub.transpose()
    da = da[["accession","copy_number"]]
    da = pd.merge(da,sub,on="accession",how="inner")
    return da

def split_test_train(da,i,multiplicand):
    X_test = da[region][i*multiplicand:(i+1)*multiplicand]
    Y_test = da['copy_number'][i*multiplicand:(i+1)*multiplicand]
    X_train = pd.concat([da[region][0:i*multiplicand],da[region][(i+1)*multiplicand:]],axis = 0)
    Y_train = pd.concat([da['copy_number'][0:i*multiplicand],da['copy_number'][(i+1)*multiplicand:]],axis = 0)
    vectorizer = CountVectorizer()
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)
    X_train = X_train.values.reshape(X_train.shape[0], )
    X_test = X_test.values.reshape(X_test.shape[0], )
    kmer_train = generate_kmer_multiple(X_train.tolist(), 6)
    kmer_test = generate_kmer_multiple(X_test.tolist(), 6)
    X_train = vectorizer.fit_transform(kmer_train).toarray()
    X_test = vectorizer.transform(kmer_test).toarray()
    return X_train,Y_train,X_test,Y_test

In [4]:
def create_model(region):
    model=Sequential()
    if region == "full-length":
        model.add(Dense(512, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64,activation='relu'))
        model.add(Dense(16,activation='relu'))
        model.add(Dense(8,activation='relu'))
        model.add(Dense(1,activation='linear'))
        model.compile(loss=root_mean_squared_error,optimizer=Adam(0.001))
    else:
        if region in ["V1-V2","V3-V4"]:
            model.add(Dense(1024, activation='relu'))
            model.add(Dense(512, activation='relu'))
            model.add(Dense(128, activation='relu'))
            model.add(Dense(64,activation='relu'))
            model.add(Dense(16,activation='relu'))
            model.add(Dense(1,activation='linear'))
        elif region in ["V4-V5"]:
            model.add(Dense(1024, activation='relu'))
            model.add(Dense(128, activation='relu'))
            model.add(Dense(64,activation='relu'))
            model.add(Dense(16,activation='relu'))
            model.add(Dense(1,activation='linear'))
        else:
            model.add(Dense(512, activation='relu'))
            if region not in ["V6-V8","V7-V9"]:
                model.add(Dense(256, activation='relu'))
            model.add(Dense(128, activation='relu'))
            model.add(Dense(64,activation='relu'))
            model.add(Dense(16,activation='relu'))
            model.add(Dense(1,activation='linear'))

        if region in ["V7-V9"]:
            model.compile(loss=root_mean_squared_error,optimizer=Adam(0.002))
        else:
            model.compile(loss=root_mean_squared_error,optimizer=Adam(0.001))

    return model

def fit_model(model,X_train,Y_train,region):
    if region == "full-length":
        model.fit(X_train,Y_train,validation_split=0.1, batch_size=100,epochs=50,verbose = 0)
    elif region == "V1-V2":
        model.fit(X_train,Y_train,validation_split=0.1,batch_size=128,epochs=50,verbose=0)
    elif region == "V1-V3":
        model.fit(X_train,Y_train,validation_split=0.1,batch_size=128,epochs=50,verbose=0)
    elif region in ["V3-V4"]:
        model.fit(X_train,Y_train,validation_split=0.1,batch_size=64,epochs=50,verbose=0)
    elif region in ["V4"]:
        model.fit(X_train,Y_train,validation_split=0.1,batch_size=100,epochs=30,verbose=0)
    elif region in ["V4-V5"]:
        model.fit(X_train,Y_train,validation_split=0.1,batch_size=64,epochs=50,verbose=0)
    elif region in ["V6-V8","V7-V9"]:
        model.fit(X_train,Y_train,validation_split=0.1,batch_size=64,epochs=50,verbose=0)
    return model

In [5]:
# V1-V2 done
# V1-V3 done
# V3-V4 done
# V4 done
# V4-V5 almost
# V6-V8 done
# V7-V9 done
performance = {}
for region in ["full-length","V1-V2","V1-V3","V3-V4","V4","V4-V5","V6-V8","V7-V9"]:
# for region in ["V4-V5"]:
    rmse = []
    da = read_region(region)
    multiplicand = int(da.shape[0]*0.2)+1
    for i in range(0,5,1):
        X_train,Y_train,X_test,Y_test=split_test_train(da,i,multiplicand)
        model = create_model(region)
        model = fit_model(model,X_train,Y_train,region)
        pred = model.predict(X_test)
        output = pd.DataFrame([pred.reshape(pred.shape[0]),Y_test],index = ["Y_pred","Y_test"])
        output = output.transpose()
        output["region"] = region
        output["group"] = i
        output.to_csv("results/results_"+region+"_"+str(i)+".csv",index = False)
        mse = mean_squared_error(Y_test, pred)
        rmse.append(sqrt(mse))
    performance[region] = rmse

2022-09-22 20:50:47.988003: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-09-22 20:50:48.088436: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3090 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 82 deviceMemorySize: 23.70GiB deviceMemoryBandwidth: 871.81GiB/s
2022-09-22 20:50:48.088484: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-09-22 20:50:48.094118: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-09-22 20:50:48.094189: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-09-22 20:50:48.095078: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcu

In [6]:
pd.DataFrame(performance).mean(axis = 0)

full-length    0.743451
V1-V2          0.797873
V1-V3          0.775106
V3-V4          0.781168
V4             0.812749
V4-V5          0.788746
V6-V8          0.785031
V7-V9          0.793758
dtype: float64

In [7]:
pd.DataFrame(performance).to_csv("MLP_method3.csv",index=False)

In [ ]:
import os
files = os.listdir("results")
results = pd.read_csv('results/results_full-length_0.csv')
for file in files[1:]:
    tmp = pd.read_csv("results/"+file)
    results = pd.concat([results,tmp],axis = 0)
results.to_csv("performance/results.csv",index=False)